### finish generating inter_gex/inter_atac

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
projdir = '/u/project/cluo/terencew/igvf/pilot/multiome/'
donors = list(np.loadtxt(f'{projdir}/txt/donors.txt', dtype=str))
samples = list(np.loadtxt(f'{projdir}/txt/samples.txt', dtype=str))
samples = samples[:2]
s = samples[0]
clusters = 4

In [3]:
donors

['25_A4', '26_A10', '27_C4', '48_V3']

In [4]:
indir = f'{projdir}/demux/regular/demuxlet/gex/'
t = samples[0]
demuxlet_gex = pd.read_csv(f'{indir}/{t}.best', sep='\t', index_col=0)
demuxlet_gex['sample'] = [t] * demuxlet_gex.shape[0]

for t in samples[1:]:
    tmp = pd.read_csv(f'{indir}/{t}.best', sep='\t', index_col=0)
    tmp['sample'] = [t] * tmp.shape[0]
    demuxlet_gex = pd.concat([demuxlet_gex, tmp], axis=0)

demuxlet_gex['t'] = [x for x in demuxlet_gex['sample']]
demuxlet_gex['barcode'] = demuxlet_gex['BARCODE']
demuxlet_gex.index = [f'{x}_{y}' for x,y in zip(demuxlet_gex['barcode'],demuxlet_gex['t'])]

d_mask = demuxlet_gex['DROPLET.TYPE'] == 'DBL'
u_mask = demuxlet_gex['DROPLET.TYPE'] == 'AMB'
demuxlet_gex.loc[d_mask,'SNG.BEST.GUESS'] = ['doublet'] * sum(d_mask)
demuxlet_gex.loc[u_mask,'SNG.BEST.GUESS'] = ['unassigned'] * sum(u_mask)

demuxlet_gex.shape

(30497, 22)

In [5]:
indir = f'{projdir}/demux/regular/demuxlet/atac/'
t = samples[0]
demuxlet_atac = pd.read_csv(f'{indir}/{t}.best', sep='\t', index_col=0)
demuxlet_atac['sample'] = [t] * demuxlet_atac.shape[0]

for t in samples[1:]:
    tmp = pd.read_csv(f'{indir}/{t}.best', sep='\t', index_col=0)
    tmp['sample'] = [t] * tmp.shape[0]
    demuxlet_atac = pd.concat([demuxlet_atac, tmp], axis=0)

demuxlet_atac['t'] = [x for x in demuxlet_atac['sample']]
demuxlet_atac['barcode'] = demuxlet_atac['BARCODE']
demuxlet_atac.index = [f'{x}_{y}' for x,y in zip(demuxlet_atac['barcode'],demuxlet_atac['t'])]

d_mask = demuxlet_atac['DROPLET.TYPE'] == 'DBL'
u_mask = demuxlet_atac['DROPLET.TYPE'] == 'AMB'
demuxlet_atac.loc[d_mask,'SNG.BEST.GUESS'] = ['doublet'] * sum(d_mask)
demuxlet_atac.loc[u_mask,'SNG.BEST.GUESS'] = ['unassigned'] * sum(u_mask)

demuxlet_atac.shape

(30497, 22)

In [6]:
# for i in range(clusters):
#     barcodes = freemuxlet[freemuxlet['SNG.BEST.GUESS'] == f'{i}'].index
#     if barcodes.empty == False:
#         ### get best assigned donor, no unassigned/doublet
#         mask = [x in donors for x in demuxlet.loc[barcodes][]'SNG.BEST.GUESS']
#         maj_donor = demuxlet.loc[barcodes][mask]['SNG.BEST.GUESS'].mode().values[0]
#         freemuxlet['SNG.BEST.GUESS'].replace({f'{i}' : f'{maj_donor}'}, inplace=True)

In [7]:
def map_free_to_anchor(free, anchor, free_col, anchor_col, clusters):
    for i in range(clusters):
        barcodes = free[free[free_col] == f'{i}'].index
        if barcodes.empty == False:
            ### get best assigned donor, no unassigned/doublet
            mask = [x in donors for x in anchor.loc[barcodes][anchor_col]]
            if not barcodes.empty:
                maj_donor = anchor.loc[barcodes][mask][anchor_col].mode().values[0]
            else:
                maj_donor = 'unassigned'
            free[free_col].replace({f'{i}' : f'{maj_donor}'}, inplace=True)
    return free

In [11]:
### because clusters are randomized, I need to assign freemuxlet at the individual level
demux_dir = f'{projdir}/demux/regular/demuxlet/gex/'
indir = f'{projdir}/demux/regular/freemuxlet/gex/'
t = samples[0]

### 
tmp_demuxlet = pd.read_csv(f'{demux_dir}/{t}.best', sep='\t', index_col=0)
tmp_demuxlet.index = tmp_demuxlet['BARCODE']
tmp_demuxlet.sort_index(inplace=True)

freemuxlet_gex = pd.read_csv(f'{indir}/{t}.clust1.samples.gz', sep='\t', index_col=0)
freemuxlet_gex.index = freemuxlet_gex['BARCODE']
freemuxlet_gex['sample'] = [t] * freemuxlet_gex.shape[0]
freemuxlet_gex['SNG.BEST.GUESS'] = [str(x) for x in freemuxlet_gex['SNG.BEST.GUESS']]
freemuxlet_gex.sort_index(inplace=True)
freemuxlet_gex = map_free_to_anchor(freemuxlet_gex, tmp_demuxlet, 'SNG.BEST.GUESS', 'SNG.BEST.GUESS', clusters)

for t in samples[1:]:
    tmp_demuxlet = pd.read_csv(f'{demux_dir}/{t}.best', sep='\t', index_col=0)
    tmp_demuxlet.index = tmp_demuxlet['BARCODE']
    tmp_demuxlet.sort_index(inplace=True)
    tmp_gex = pd.read_csv(f'{indir}/{t}.clust1.samples.gz', sep='\t', index_col=0)
    tmp_gex.index = tmp_gex['BARCODE']
    tmp_gex['sample'] = [t] * tmp_gex.shape[0]
    tmp_gex['SNG.BEST.GUESS'] = [str(x) for x in tmp_gex['SNG.BEST.GUESS']]
    tmp_gex.sort_index(inplace=True)

    tmp_gex = map_free_to_anchor(tmp_gex, tmp_demuxlet, 'SNG.BEST.GUESS', 'SNG.BEST.GUESS', clusters)
    freemuxlet_gex = pd.concat([freemuxlet_gex, tmp_gex], axis=0)

freemuxlet_gex['t'] = [x for x in freemuxlet_gex['sample']]
freemuxlet_gex['barcode'] = freemuxlet_gex['BARCODE']
freemuxlet_gex.index = [f'{x}_{y}' for x,y in zip(freemuxlet_gex['barcode'],freemuxlet_gex['t'])]

d_mask = freemuxlet_gex['DROPLET.TYPE'] == 'DBL'
u_mask = freemuxlet_gex['DROPLET.TYPE'] == 'AMB'
freemuxlet_gex.loc[d_mask,'SNG.BEST.GUESS'] = ['doublet'] * sum(d_mask)
freemuxlet_gex.loc[u_mask,'SNG.BEST.GUESS'] = ['unassigned'] * sum(u_mask)

freemuxlet_gex.shape

(30497, 22)

In [12]:
freemuxlet_gex['SNG.BEST.GUESS'].value_counts()

25_A4         10010
27_C4          6428
48_V3          6349
26_A10         5385
doublet        2186
unassigned      139
Name: SNG.BEST.GUESS, dtype: int64

In [13]:
!ls {indir}

20220928-IGVF-D0.clust0.samples.gz  20220928-IGVF-D3.clust1.vcf.gz
20220928-IGVF-D0.clust0.vcf.gz	    20220928-IGVF-D3.lmix
20220928-IGVF-D0.clust1.samples.gz  20220928-IGVF-D7.clust0.samples.gz
20220928-IGVF-D0.clust1.vcf.gz	    20220928-IGVF-D7.clust0.vcf.gz
20220928-IGVF-D0.lmix		    20220928-IGVF-D7.clust1.samples.gz
20220928-IGVF-D3.clust0.samples.gz  20220928-IGVF-D7.clust1.vcf.gz
20220928-IGVF-D3.clust0.vcf.gz	    20220928-IGVF-D7.lmix
20220928-IGVF-D3.clust1.samples.gz


In [14]:
### because clusters are randomized, I need to assign freemuxlet at the individual level
demux_dir = f'{projdir}/demux/regular/demuxlet/atac/'
indir = f'{projdir}/demux/regular/freemuxlet/atac/'
t = samples[0]

### 
tmp_demuxlet = pd.read_csv(f'{demux_dir}/{t}.best', sep='\t', index_col=0)
tmp_demuxlet.index = tmp_demuxlet['BARCODE']
tmp_demuxlet.sort_index(inplace=True)

freemuxlet_atac = pd.read_csv(f'{indir}/{t}.clust1.samples.gz', sep='\t', index_col=0)
freemuxlet_atac.index = freemuxlet_atac['BARCODE']
freemuxlet_atac['sample'] = [t] * freemuxlet_atac.shape[0]
freemuxlet_atac['SNG.BEST.GUESS'] = [str(x) for x in freemuxlet_atac['SNG.BEST.GUESS']]
freemuxlet_atac.sort_index(inplace=True)
freemuxlet_atac = map_free_to_anchor(freemuxlet_atac, tmp_demuxlet, 'SNG.BEST.GUESS', 'SNG.BEST.GUESS', clusters)

for t in samples[1:]:
    tmp_demuxlet = pd.read_csv(f'{demux_dir}/{t}.best', sep='\t', index_col=0)
    tmp_demuxlet.index = tmp_demuxlet['BARCODE']
    tmp_demuxlet.sort_index(inplace=True)
    tmp_atac = pd.read_csv(f'{indir}/{t}.clust1.samples.gz', sep='\t', index_col=0)
    tmp_atac.index = tmp_atac['BARCODE']
    tmp_atac['sample'] = [t] * tmp_atac.shape[0]
    tmp_atac['SNG.BEST.GUESS'] = [str(x) for x in tmp_atac['SNG.BEST.GUESS']]
    tmp_atac.sort_index(inplace=True)

    tmp_atac = map_free_to_anchor(tmp_atac, tmp_demuxlet, 'SNG.BEST.GUESS', 'SNG.BEST.GUESS', clusters)
    freemuxlet_atac = pd.concat([freemuxlet_atac, tmp_atac], axis=0)

freemuxlet_atac['t'] = [x for x in freemuxlet_atac['sample']]
freemuxlet_atac['barcode'] = freemuxlet_atac['BARCODE']
freemuxlet_atac.index = [f'{x}_{y}' for x,y in zip(freemuxlet_atac['barcode'],freemuxlet_atac['t'])]

d_mask = freemuxlet_atac['DROPLET.TYPE'] == 'DBL'
u_mask = freemuxlet_atac['DROPLET.TYPE'] == 'AMB'
freemuxlet_atac.loc[d_mask,'SNG.BEST.GUESS'] = ['doublet'] * sum(d_mask)
freemuxlet_atac.loc[u_mask,'SNG.BEST.GUESS'] = ['unassigned'] * sum(u_mask)

freemuxlet_atac.shape

(30497, 22)

In [15]:
freemuxlet_atac['SNG.BEST.GUESS'].value_counts()

25_A4         9882
48_V3         7032
27_C4         6588
26_A10        5511
doublet       1476
unassigned       8
Name: SNG.BEST.GUESS, dtype: int64

In [16]:
indir = f'{projdir}/demux/regular/vireo/gex/'
t = samples[0]
vireo_gex = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
vireo_gex['sample'] = [t] * vireo_gex.shape[0]

for t in samples[1:]:
    tmp = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp['sample'] = [t] * tmp.shape[0]
    vireo_gex = pd.concat([vireo_gex, tmp], axis=0)

vireo_gex['t'] = [x for x in vireo_gex['sample']]
vireo_gex['barcode'] = vireo_gex.index
vireo_gex.index = [f'{x}_{y}' for x,y in zip(vireo_gex['barcode'],vireo_gex['t'])]

vireo_gex.shape

(30497, 10)

In [17]:
indir = f'{projdir}/demux/regular/vireo/atac/'
t = samples[0]
vireo_atac = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
vireo_atac['sample'] = [t] * vireo_atac.shape[0]

for t in samples[1:]:
    tmp = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp['sample'] = [t] * tmp.shape[0]
    vireo_atac = pd.concat([vireo_atac, tmp], axis=0)

vireo_atac['t'] = [x for x in vireo_atac['sample']]
vireo_atac['barcode'] = vireo_atac.index
vireo_atac.index = [f'{x}_{y}' for x,y in zip(vireo_atac['barcode'],vireo_atac['t'])]

vireo_atac.shape

(30497, 10)

In [18]:
vireo_dir = f'{projdir}/demux/regular/vireo/gex/'

indir = f'{projdir}/demux/regular/vireo_nogenos/gex/'
t = samples[0]
tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
vireo_nogenos_gex = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
vireo_nogenos_gex['sample'] = [t] * vireo_nogenos_gex.shape[0]
vireo_nogenos_gex['donor_id'] = [x.replace('donor', '') for x in vireo_nogenos_gex['donor_id']]
vireo_nogenos_gex = map_free_to_anchor(vireo_nogenos_gex, tmp_vireo, 'donor_id', 'donor_id', clusters)

for t in samples[1:]:
    tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_gex = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_gex['sample'] = [t] * tmp_gex.shape[0]
    tmp_gex['donor_id'] = [x.replace('donor', '') for x in tmp_gex['donor_id']]
    tmp_gex = map_free_to_anchor(tmp_gex, tmp_vireo, 'donor_id', 'donor_id', clusters)
    vireo_nogenos_gex = pd.concat([vireo_nogenos_gex, tmp_gex], axis=0)

vireo_nogenos_gex['t'] = [x for x in vireo_nogenos_gex['sample']]
vireo_nogenos_gex['barcode'] = vireo_nogenos_gex.index
vireo_nogenos_gex.index = [f'{x}_{y}' for x,y in zip(vireo_nogenos_gex['barcode'],vireo_nogenos_gex['t'])]

vireo_nogenos_gex.shape

(30497, 10)

In [19]:
vireo_nogenos_gex['donor_id'].value_counts()

25_A4         9064
48_V3         5025
27_C4         4933
26_A10        4327
doublet       4192
unassigned    2956
Name: donor_id, dtype: int64

In [20]:
vireo_dir = f'{projdir}/demux/regular/vireo/atac/'

indir = f'{projdir}/demux/regular/vireo_nogenos/atac/'
t = samples[0]
tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
vireo_nogenos_atac = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
vireo_nogenos_atac['sample'] = [t] * vireo_nogenos_atac.shape[0]
vireo_nogenos_atac['donor_id'] = [x.replace('donor', '') for x in vireo_nogenos_atac['donor_id']]
vireo_nogenos_atac = map_free_to_anchor(vireo_nogenos_atac, tmp_vireo, 'donor_id', 'donor_id', clusters)

for t in samples[1:]:
    tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_atac = pd.read_csv(f'{indir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_atac['sample'] = [t] * tmp_atac.shape[0]
    tmp_atac['donor_id'] = [x.replace('donor', '') for x in tmp_atac['donor_id']]
    tmp_atac = map_free_to_anchor(tmp_atac, tmp_vireo, 'donor_id', 'donor_id', clusters)
    vireo_nogenos_atac = pd.concat([vireo_nogenos_atac, tmp_atac], axis=0)

vireo_nogenos_atac['t'] = [x for x in vireo_nogenos_atac['sample']]
vireo_nogenos_atac['barcode'] = vireo_nogenos_atac.index
vireo_nogenos_atac.index = [f'{x}_{y}' for x,y in zip(vireo_nogenos_atac['barcode'],vireo_nogenos_atac['t'])]

vireo_nogenos_atac.shape

(30497, 10)

In [21]:
vireo_nogenos_atac['donor_id'].value_counts()

25_A4         9019
48_V3         6058
27_C4         5350
doublet       5011
26_A10        4465
unassigned     594
Name: donor_id, dtype: int64

In [22]:
vireo_dir = f'{projdir}/demux/regular/vireo/gex/'

indir = f'{projdir}/demux/regular/souporcell/gex/'
t = samples[0]
tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
souporcell_gex = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
souporcell_gex['sample'] = [t] * souporcell_gex.shape[0]
souporcell_gex = map_free_to_anchor(souporcell_gex, tmp_vireo, 'assignment', 'donor_id', clusters)

for t in samples[1:]:
    tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_gex = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
    tmp_gex['sample'] = [t] * tmp_gex.shape[0]
    tmp_gex = map_free_to_anchor(tmp_gex, tmp_vireo, 'assignment', 'donor_id', clusters)
    souporcell_gex = pd.concat([souporcell_gex, tmp_gex], axis=0)

souporcell_gex['t'] = [x for x in souporcell_gex['sample']]
souporcell_gex['barcode'] = souporcell_gex.index
souporcell_gex.index = [f'{x}_{y}' for x,y in zip(souporcell_gex['barcode'],souporcell_gex['t'])]

d_mask = souporcell_gex['status'] == 'doublet'
u_mask = souporcell_gex['status'] == 'unassigned'
souporcell_gex.loc[d_mask,'assignment'] = ['doublet'] * sum(d_mask)
souporcell_gex.loc[u_mask,'assignment'] = ['unassigned'] * sum(u_mask)

souporcell_gex.shape

(30497, 11)

In [23]:
vireo_dir = f'{projdir}/demux/regular/vireo/atac/'

indir = f'{projdir}/demux/regular/souporcell/atac/'
t = samples[0]
tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
souporcell_atac = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
souporcell_atac['sample'] = [t] * souporcell_atac.shape[0]
souporcell_atac = map_free_to_anchor(souporcell_atac, tmp_vireo, 'assignment', 'donor_id', clusters)

for t in samples[1:]:
    tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_atac = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
    tmp_atac['sample'] = [t] * tmp_atac.shape[0]
    tmp_atac = map_free_to_anchor(tmp_atac, tmp_vireo, 'assignment', 'donor_id', clusters)
    souporcell_atac = pd.concat([souporcell_atac, tmp_atac], axis=0)

souporcell_atac['t'] = [x for x in souporcell_atac['sample']]
souporcell_atac['barcode'] = souporcell_atac.index
souporcell_atac.index = [f'{x}_{y}' for x,y in zip(souporcell_atac['barcode'],souporcell_atac['t'])]

d_mask = souporcell_atac['status'] == 'doublet'
u_mask = souporcell_atac['status'] == 'unassigned'
souporcell_atac.loc[d_mask,'assignment'] = ['doublet'] * sum(d_mask)
souporcell_atac.loc[u_mask,'assignment'] = ['unassigned'] * sum(u_mask)

souporcell_atac.shape

(30497, 11)

In [24]:
vireo_dir = f'{projdir}/demux/regular/vireo/gex/'

indir = f'{projdir}/demux/regular/souporcell_nogenos/gex/'
t = samples[0]
tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
souporcell_nogenos_gex = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
souporcell_nogenos_gex['sample'] = [t] * souporcell_nogenos_gex.shape[0]
souporcell_nogenos_gex = map_free_to_anchor(souporcell_nogenos_gex, tmp_vireo, 'assignment', 'donor_id', clusters)

for t in samples[1:]:
    tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_gex = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
    tmp_gex['sample'] = [t] * tmp_gex.shape[0]
    tmp_gex = map_free_to_anchor(tmp_gex, tmp_vireo, 'assignment', 'donor_id', clusters)
    souporcell_nogenos_gex = pd.concat([souporcell_nogenos_gex, tmp_gex], axis=0)

souporcell_nogenos_gex['t'] = [x for x in souporcell_nogenos_gex['sample']]
souporcell_nogenos_gex['barcode'] = souporcell_nogenos_gex.index
souporcell_nogenos_gex.index = [f'{x}_{y}' for x,y in zip(souporcell_nogenos_gex['barcode'],souporcell_nogenos_gex['t'])]

d_mask = souporcell_nogenos_gex['status'] == 'doublet'
u_mask = souporcell_nogenos_gex['status'] == 'unassigned'
souporcell_nogenos_gex.loc[d_mask,'assignment'] = ['doublet'] * sum(d_mask)
souporcell_nogenos_gex.loc[u_mask,'assignment'] = ['unassigned'] * sum(u_mask)

souporcell_nogenos_gex.shape

(30497, 11)

In [25]:
souporcell_gex['assignment'].value_counts()

25_A4         9509
26_A10        5107
27_C4         5026
doublet       4854
48_V3         4496
unassigned    1505
Name: assignment, dtype: int64

In [26]:
vireo_dir = f'{projdir}/demux/regular/vireo/atac/'

indir = f'{projdir}/demux/regular/souporcell_nogenos/atac/'
t = samples[0]
tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
souporcell_nogenos_atac = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
souporcell_nogenos_atac['sample'] = [t] * souporcell_nogenos_atac.shape[0]
souporcell_nogenos_atac = map_free_to_anchor(souporcell_nogenos_atac, tmp_vireo, 'assignment', 'donor_id', clusters)

for t in samples[1:]:
    tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    tmp_atac = pd.read_csv(f'{indir}/{t}/clusters.tsv', sep='\t', index_col=0)
    tmp_atac['sample'] = [t] * tmp_atac.shape[0]
    tmp_atac = map_free_to_anchor(tmp_atac, tmp_vireo, 'assignment', 'donor_id', clusters)
    souporcell_nogenos_atac = pd.concat([souporcell_nogenos_atac, tmp_atac], axis=0)

souporcell_nogenos_atac['t'] = [x for x in souporcell_nogenos_atac['sample']]
souporcell_nogenos_atac['barcode'] = souporcell_nogenos_atac.index
souporcell_nogenos_atac.index = [f'{x}_{y}' for x,y in zip(souporcell_nogenos_atac['barcode'],souporcell_nogenos_atac['t'])]

d_mask = souporcell_nogenos_atac['status'] == 'doublet'
u_mask = souporcell_nogenos_atac['status'] == 'unassigned'
souporcell_nogenos_atac.loc[d_mask,'assignment'] = ['doublet'] * sum(d_mask)
souporcell_nogenos_atac.loc[u_mask,'assignment'] = ['unassigned'] * sum(u_mask)

souporcell_nogenos_atac.shape

(30497, 11)

In [27]:
souporcell_atac['assignment'].value_counts()

25_A4         9031
48_V3         5939
27_C4         5534
26_A10        5104
doublet       4723
unassigned     166
Name: assignment, dtype: int64

In [28]:
indir = f'{projdir}/demux/regular/demuxalot/gex'
t = samples[0]
demuxalot_gex = pd.read_csv(f'{indir}/{t}/{t}_posteriors.csv', comment='#', sep='\t', index_col=0)
demuxalot_gex[f'assignment'] = demuxalot_gex.idxmax(axis=1)
demuxalot_gex['doublet_status'] = ['+' in x for x in demuxalot_gex[f'assignment']]
mask = demuxalot_gex['doublet_status']
demuxalot_gex.loc[mask,f'assignment'] = 'doublet'
demuxalot_gex['sample'] = [t] * demuxalot_gex.shape[0]

for t in samples[1:]:
    tmp = pd.read_csv(f'{indir}/{t}/{t}_posteriors.csv', comment='#', sep='\t', index_col=0)
    tmp[f'assignment'] = tmp.idxmax(axis=1)
    tmp['doublet_status'] = ['+' in x for x in tmp[f'assignment']]
    mask = tmp['doublet_status']
    tmp.loc[mask,f'assignment'] = 'doublet'
    tmp['sample'] = [t] * tmp.shape[0]
    demuxalot_gex = pd.concat([demuxalot_gex, tmp], axis=0)

demuxalot_gex['barcode'] = demuxalot_gex.index
demuxalot_gex.index = [f'{x}_{y}' for x,y in zip(demuxalot_gex['barcode'],demuxalot_gex['sample'])]
    
demuxalot_gex.shape    

(30497, 14)

In [29]:
!ls {projdir}/demux/regular/scsplit/gex/*-D0/out

scSplit.log  scSplit_P_s_c.csv	scSplit_result.csv


In [125]:
vireo_dir = f'{projdir}/demux/regular/vireo/gex'
indir = f'{projdir}/demux/regular/scsplit/gex'

t = samples[0]
tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
barcodes = pd.read_csv(f'{indir}/{t}/out/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
scsplit_gex = pd.DataFrame(index=barcodes)
scsplit_gex['donor_id'] = 'unassigned'

results = pd.read_csv(f'{indir}/{t}/out/scSplit_result.csv', comment='#', sep='\t', index_col=0)
b_mask = scsplit_gex.index.isin(results.index)
scsplit_gex.loc[b_mask,'donor_id'] = results['Cluster']
scsplit_gex['donor_id'] = [str(x).replace('SNG-', '') for x in scsplit_gex['donor_id']]
d_mask = ['DBL' in x for x in scsplit_gex['donor_id']]
scsplit_gex.loc[d_mask,'donor_id'] = 'doublet'
scsplit_gex = map_free_to_anchor(scsplit_gex, tmp_vireo, 'donor_id', 'donor_id', clusters+1)
scsplit_gex['t'] = t

for t in samples[1:]:
    print(t)
    tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    barcodes = pd.read_csv(f'{indir}/{t}/out/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
    tmp_scsplit = pd.DataFrame(index=barcodes)
    tmp_scsplit['donor_id'] = 'unassigned'

    results = pd.read_csv(f'{indir}/{t}/out/scSplit_result.csv', comment='#', sep='\t', index_col=0)
    b_mask = tmp_scsplit.index.isin(results.index)
    
    tmp_scsplit.loc[b_mask,'donor_id'] = results['Cluster']
    tmp_scsplit['donor_id'] = [str(x).replace('SNG-', '') for x in tmp_scsplit['donor_id']]
    d_mask = ['DBL' in x for x in tmp_scsplit['donor_id']]
    tmp_scsplit.loc[d_mask,'donor_id'] = 'doublet'
    tmp_scsplit = map_free_to_anchor(tmp_scsplit, tmp_vireo, 'donor_id', 'donor_id', clusters+1)
    tmp_scsplit['t'] = t

    scsplit_gex = pd.concat([scsplit_gex, tmp_scsplit], axis=0)
    
scsplit_gex.index = [f'{x}_{y}' for x,y in zip(scsplit_gex.index,scsplit_gex['t'])]
scsplit_gex.shape

# tmp_scsplit_gex = vireo_gex.copy()
# ### randomly change 10% of it idk lol
# tmp_scsplit_gex.

20220928-IGVF-D3


(30497, 2)

In [126]:
tmp_scsplit.shape

(21010, 2)

In [127]:
tmp_scsplit

,donor_id,t
AAACAGCCAAAGGCCA-1,27_C4,20220928-IGVF-D3
AAACAGCCAACATAAG-1,25_A4,20220928-IGVF-D3
AAACAGCCAATTAGGA-1,48_V3,20220928-IGVF-D3
AAACAGCCACCCACAG-1,26_A10,20220928-IGVF-D3
AAACAGCCAGCAATAA-1,25_A4,20220928-IGVF-D3
...,...,...
TTTGTTGGTTCCAGGG-1,26_A10,20220928-IGVF-D3
TTTGTTGGTTCCTGTG-1,25_A4,20220928-IGVF-D3
TTTGTTGGTTGCCTCA-1,25_A4,20220928-IGVF-D3
TTTGTTGGTTGGATAT-1,25_A4,20220928-IGVF-D3


In [128]:
vireo_dir = f'{projdir}/demux/regular/vireo/atac'
indir = f'{projdir}/demux/regular/scsplit/atac'

t = samples[0]
tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
barcodes = pd.read_csv(f'{indir}/{t}/out/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
scsplit_atac = pd.DataFrame(index=barcodes)
scsplit_atac['donor_id'] = 'unassigned'

results = pd.read_csv(f'{indir}/{t}/out/scSplit_result.csv', comment='#', sep='\t', index_col=0)
b_mask = scsplit_atac.index.isin(results.index)
scsplit_atac.loc[b_mask,'donor_id'] = results['Cluster']
scsplit_atac['donor_id'] = [str(x).replace('SNG-', '') for x in scsplit_atac['donor_id']]
d_mask = ['DBL' in x for x in scsplit_atac['donor_id']]
scsplit_atac.loc[d_mask,'donor_id'] = 'doublet'
scsplit_atac = map_free_to_anchor(scsplit_atac, tmp_vireo, 'donor_id', 'donor_id', clusters+1)
scsplit_atac['t'] = t

for t in samples[1:]:
    print(t)
    tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    barcodes = pd.read_csv(f'{indir}/{t}/out/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
    tmp_scsplit = pd.DataFrame(index=barcodes)
    tmp_scsplit['donor_id'] = 'unassigned'

    results = pd.read_csv(f'{indir}/{t}/out/scSplit_result.csv', comment='#', sep='\t', index_col=0)
    b_mask = tmp_scsplit.index.isin(results.index)
    tmp_scsplit.loc[b_mask,'donor_id'] = results['Cluster']
    tmp_scsplit['donor_id'] = [str(x).replace('SNG-', '') for x in tmp_scsplit['donor_id']]
    d_mask = ['DBL' in x for x in tmp_scsplit['donor_id']]
    tmp_scsplit.loc[d_mask,'donor_id'] = 'doublet'
    tmp_scsplit = map_free_to_anchor(tmp_scsplit, tmp_vireo, 'donor_id', 'donor_id', clusters+1)
    tmp_scsplit['t'] = t
    
    scsplit_atac = pd.concat([scsplit_atac, tmp_scsplit], axis=0)

    
scsplit_atac.index = [f'{x}_{y}' for x,y in zip(scsplit_atac.index,scsplit_atac['t'])]
scsplit_atac.shape

# tmp_scsplit_atac = vireo_atac.copy()
# ### randomly change 10% of it idk lol
# tmp_scsplit_atac.

20220928-IGVF-D3


(30497, 2)

In [129]:
!ls {indir}/{t}/

20220928-IGVF-D3.filtered.vcf	      20220928-IGVF-D3.vcf  varcon_cellsnp
20220928-IGVF-D3.filtered.vcf.gz      cellsnp
20220928-IGVF-D3.filtered.vcf.gz.tbi  out


In [130]:
vireo_dir = f'{projdir}/demux/regular/vireo/gex'
indir = f'{projdir}/demux/regular/scsplit_nogenos/gex/'

t = samples[0]
tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
barcodes = pd.read_csv(f'{indir}/{t}/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
scsplit_nogenos_gex = pd.DataFrame(index=barcodes)
scsplit_nogenos_gex['donor_id'] = 'unassigned'

results = pd.read_csv(f'{indir}/{t}/scSplit_result.csv', comment='#', sep='\t', index_col=0)
b_mask = scsplit_nogenos_gex.index.isin(results.index)
scsplit_nogenos_gex.loc[b_mask,'donor_id'] = results['Cluster']
scsplit_nogenos_gex['donor_id'] = [str(x).replace('SNG-', '') for x in scsplit_nogenos_gex['donor_id']]
d_mask = ['DBL' in x for x in scsplit_nogenos_gex['donor_id']]
scsplit_nogenos_gex.loc[d_mask,'donor_id'] = 'doublet'
scsplit_nogenos_gex = map_free_to_anchor(scsplit_nogenos_gex, tmp_vireo, 'donor_id', 'donor_id', clusters+1)
scsplit_nogenos_gex['t'] = t

for t in samples[1:]:
    print(t)
    tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    barcodes = pd.read_csv(f'{indir}/{t}/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
    tmp_scsplit_nogenos = pd.DataFrame(index=barcodes)
    tmp_scsplit_nogenos['donor_id'] = 'unassigned'

    results = pd.read_csv(f'{indir}/{t}/scSplit_result.csv', comment='#', sep='\t', index_col=0)
    b_mask = tmp_scsplit_nogenos.index.isin(results.index)
    tmp_scsplit_nogenos.loc[b_mask,'donor_id'] = results['Cluster']
    tmp_scsplit_nogenos['donor_id'] = [str(x).replace('SNG-', '') for x in tmp_scsplit_nogenos['donor_id']]
    d_mask = ['DBL' in x for x in tmp_scsplit_nogenos['donor_id']]
    tmp_scsplit_nogenos.loc[d_mask,'donor_id'] = 'doublet'
    tmp_scsplit_nogenos = map_free_to_anchor(tmp_scsplit_nogenos, tmp_vireo, 'donor_id', 'donor_id', clusters+1)
    tmp_scsplit_nogenos['t'] = t
    
    scsplit_nogenos_gex = pd.concat([scsplit_nogenos_gex, tmp_scsplit_nogenos], axis=0)


scsplit_nogenos_gex.index = [f'{x}_{y}' for x,y in zip(scsplit_nogenos_gex.index,scsplit_nogenos_gex['t'])]
scsplit_nogenos_gex.shape[0]

20220928-IGVF-D3


30497

In [131]:
vireo_dir = f'{projdir}/demux/regular/vireo/atac'
indir = f'{projdir}/demux/regular/scsplit_nogenos/atac/'

t = samples[0]
tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
barcodes = pd.read_csv(f'{indir}/{t}/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
scsplit_nogenos_atac = pd.DataFrame(index=barcodes)
scsplit_nogenos_atac['donor_id'] = 'unassigned'

results = pd.read_csv(f'{indir}/{t}/scSplit_result.csv', comment='#', sep='\t', index_col=0)
b_mask = scsplit_nogenos_atac.index.isin(results.index)
scsplit_nogenos_atac.loc[b_mask,'donor_id'] = results['Cluster']
scsplit_nogenos_atac['donor_id'] = [str(x).replace('SNG-', '') for x in scsplit_nogenos_atac['donor_id']]
d_mask = ['DBL' in x for x in scsplit_nogenos_atac['donor_id']]
scsplit_nogenos_atac.loc[d_mask,'donor_id'] = 'doublet'
scsplit_nogenos_atac = map_free_to_anchor(scsplit_nogenos_atac, tmp_vireo, 'donor_id', 'donor_id', clusters+1)
scsplit_nogenos_atac['t'] = t

for t in samples[1:]:
    print(t)
    tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
    barcodes = pd.read_csv(f'{indir}/{t}/scSplit_P_s_c.csv', sep=',', header=0, index_col=0).index
    tmp_scsplit_nogenos = pd.DataFrame(index=barcodes)
    tmp_scsplit_nogenos['donor_id'] = 'unassigned'

    results = pd.read_csv(f'{indir}/{t}/scSplit_result.csv', comment='#', sep='\t', index_col=0)
    b_mask = tmp_scsplit_nogenos.index.isin(results.index)
    tmp_scsplit_nogenos.loc[b_mask,'donor_id'] = results['Cluster']
    tmp_scsplit_nogenos['donor_id'] = [str(x).replace('SNG-', '') for x in tmp_scsplit_nogenos['donor_id']]
    d_mask = ['DBL' in x for x in tmp_scsplit_nogenos['donor_id']]
    tmp_scsplit_nogenos.loc[d_mask,'donor_id'] = 'doublet'
    tmp_scsplit_nogenos = map_free_to_anchor(tmp_scsplit_nogenos, tmp_vireo, 'donor_id', 'donor_id', clusters+1)
    tmp_scsplit_nogenos['t'] = t
    
    scsplit_nogenos_atac = pd.concat([scsplit_nogenos_atac, tmp_scsplit_nogenos], axis=0)


scsplit_nogenos_atac.index = [f'{x}_{y}' for x,y in zip(scsplit_nogenos_atac.index,scsplit_nogenos_atac['t'])]
scsplit_nogenos_atac.shape[0]

20220928-IGVF-D3


30497

In [135]:
scsplit_gex['donor_id'].value_counts(), \
scsplit_atac['donor_id'].value_counts(), \
scsplit_nogenos_gex['donor_id'].value_counts(), \
scsplit_nogenos_atac['donor_id'].value_counts()

(25_A4         9386
 48_V3         6234
 26_A10        5239
 27_C4         5102
 doublet       2825
 unassigned    1711
 Name: donor_id, dtype: int64,
 25_A4         8630
 48_V3         5630
 27_C4         4942
 doublet       4925
 unassigned    3187
 26_A10        3183
 Name: donor_id, dtype: int64,
 25_A4         9119
 27_C4         5900
 26_A10        5321
 doublet       5225
 48_V3         3230
 unassigned    1702
 Name: donor_id, dtype: int64,
 25_A4         9282
 48_V3         7258
 27_C4         6567
 26_A10        5101
 doublet       2103
 unassigned     186
 Name: donor_id, dtype: int64)

In [136]:
### no idea what happened
# vireo_dir = f'{projdir}/demux/regular/vireo/atac/'

# indir = f'{projdir}/demux/regular/scavengers/atac/'
# t = samples[0]
# tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
# scavengers_atac = pd.read_csv(f'{indir}/{t}/clusters.final.tsv', sep='\t')
# scavengers_atac['assignment'] = scavengers_atac['barcode']
# scavengers_atac['barcode'] = [x[0] for x in scavengers_atac.index]
# scavengers_atac['status'] = [x[1] for x in scavengers_atac.index]
# scavengers_atac.index = scavengers_atac['barcode']

# d_mask = scavengers_atac['status'] == 'doublet'
# u_mask = scavengers_atac['status'] == 'unassigned'
# scavengers_atac.loc[d_mask,'assignment'] = ['doublet'] * sum(d_mask)
# scavengers_atac.loc[u_mask,'assignment'] = ['unassigned'] * sum(u_mask)
# scavengers_atac['sample'] = t
# scavengers_atac = map_free_to_anchor(scavengers_atac, tmp_vireo, 'assignment', 'donor_id', clusters)

# for t in samples[1:]:
#     tmp_vireo = pd.read_csv(f'{vireo_dir}/{t}/donor_ids.tsv', sep='\t', index_col=0)
#     tmp_atac = pd.read_csv(f'{indir}/{t}/clusters.final.tsv', sep='\t')
#     tmp_atac['assignment'] = tmp_atac['barcode']
#     tmp_atac['barcode'] = [x[0] for x in tmp_atac.index]
#     tmp_atac['status'] = [x[1] for x in tmp_atac.index]
#     tmp_atac.index = tmp_atac['barcode']
#     tmp_atac['t'] = t

#     d_mask = tmp_atac['status'] == 'doublet'
#     u_mask = tmp_atac['status'] == 'unassigned'
#     tmp_atac.loc[d_mask,'assignment'] = ['doublet'] * sum(d_mask)
#     tmp_atac.loc[u_mask,'assignment'] = ['unassigned'] * sum(u_mask)
#     tmp_atac = map_free_to_anchor(tmp_atac, tmp_vireo, 'assignment', 'donor_id', clusters)
    
#     tmp_atac['sample'] = t
#     scavengers_atac = pd.concat([scavengers_atac, tmp_atac], axis=0)

# scavengers_atac['t'] = [x for x in scavengers_atac['sample']]
# scavengers_atac['barcode'] = scavengers_atac.index
# scavengers_atac.index = [f'{x}_{y}' for x,y in zip(scavengers_atac['barcode'],scavengers_atac['t'])]

# d_mask = scavengers_atac['status'] == 'doublet'
# u_mask = scavengers_atac['status'] == 'unassigned'
# scavengers_atac.loc[d_mask,'assignment'] = ['doublet'] * sum(d_mask)
# scavengers_atac.loc[u_mask,'assignment'] = ['unassigned'] * sum(u_mask)

# scavengers_atac.shape

In [137]:
scavengers_atac = pd.read_csv(f'{projdir}/csv/demux/atac/scavengers.csv', sep='\t', header=0, index_col=0)
scavengers_atac.shape

(30497, 11)

In [138]:
demuxlet_gex.to_csv(f'{projdir}/csv/demux/gex/demuxlet.csv', sep='\t', header=True, index=True)
freemuxlet_gex.to_csv(f'{projdir}/csv/demux/gex/freemuxlet.csv', sep='\t', header=True, index=True)
vireo_gex.to_csv(f'{projdir}/csv/demux/gex/vireo.csv', sep='\t', header=True, index=True)
vireo_nogenos_gex.to_csv(f'{projdir}/csv/demux/gex/vireo_nogenos.csv', sep='\t', header=True, index=True)
souporcell_gex.to_csv(f'{projdir}/csv/demux/gex/souporcell.csv', sep='\t', header=True, index=True)
souporcell_nogenos_gex.to_csv(f'{projdir}/csv/demux/gex/souporcell_nogenos.csv', sep='\t', header=True, index=True)
demuxalot_gex.to_csv(f'{projdir}/csv/demux/gex/demuxalot.csv', sep='\t', header=True, index=True)
scsplit_gex.to_csv(f'{projdir}/csv/demux/gex/scsplit.csv', sep='\t', header=True, index=True)
scsplit_nogenos_gex.to_csv(f'{projdir}/csv/demux/gex/scsplit_nogenos.csv', sep='\t', header=True, index=True)

In [139]:
demuxlet_atac.to_csv(f'{projdir}/csv/demux/atac/demuxlet.csv', sep='\t', header=True, index=True)
freemuxlet_atac.to_csv(f'{projdir}/csv/demux/atac/freemuxlet.csv', sep='\t', header=True, index=True)
vireo_atac.to_csv(f'{projdir}/csv/demux/atac/vireo.csv', sep='\t', header=True, index=True)
vireo_nogenos_atac.to_csv(f'{projdir}/csv/demux/atac/vireo_nogenos.csv', sep='\t', header=True, index=True)
souporcell_atac.to_csv(f'{projdir}/csv/demux/atac/souporcell.csv', sep='\t', header=True, index=True)
souporcell_nogenos_atac.to_csv(f'{projdir}/csv/demux/atac/souporcell_nogenos.csv', sep='\t', header=True, index=True)
scsplit_atac.to_csv(f'{projdir}/csv/demux/atac/scsplit.csv', sep='\t', header=True, index=True)
scsplit_nogenos_atac.to_csv(f'{projdir}/csv/demux/atac/scsplit_nogenos.csv', sep='\t', header=True, index=True)
scavengers_atac.to_csv(f'{projdir}/csv/demux/atac/scavengers.csv', sep='\t', header=True, index=True)

### praying for day 3 to work  eventually

### create dataframe
let another notebook handle the conversion between clusters to the actual donors

In [140]:
inter_gex = pd.DataFrame(index=vireo_gex.index)
inter_gex['demuxlet'] = demuxlet_gex['SNG.BEST.GUESS']
inter_gex['freemuxlet'] = freemuxlet_gex['SNG.BEST.GUESS']
inter_gex['vireo'] = vireo_gex['donor_id']
inter_gex['vireo_nogenos'] = vireo_nogenos_gex['donor_id']
inter_gex['souporcell'] = souporcell_gex['assignment']
inter_gex['souporcell_nogenos'] = souporcell_nogenos_gex['assignment']
inter_gex['demuxalot'] = demuxalot_gex['assignment']
inter_gex['scsplit'] = scsplit_gex['donor_id']
inter_gex['scsplit_nogenos'] = scsplit_nogenos_gex['donor_id']

In [141]:
inter_atac = pd.DataFrame(index=vireo_atac.index)
inter_atac['demuxlet'] = demuxlet_atac['SNG.BEST.GUESS']
inter_atac['freemuxlet'] = freemuxlet_atac['SNG.BEST.GUESS']
inter_atac['vireo'] = vireo_atac['donor_id']
inter_atac['vireo_nogenos'] = vireo_nogenos_atac['donor_id']
inter_atac['souporcell'] = souporcell_atac['assignment']
inter_atac['souporcell_nogenos'] = souporcell_nogenos_atac['assignment']
inter_atac['scsplit'] = scsplit_atac['donor_id']
inter_atac['scsplit_nogenos'] = scsplit_nogenos_atac['donor_id']
inter_atac['scavengers'] = scavengers_atac['assignment']

In [142]:
# tmp_gex = inter_gex.copy()
# mask = [x.split('_')[1] == samples[1] for x in tmp_gex.index]
# tmp_gex = tmp_gex[mask]
# tmp_gex['scsplit'] = tmp_gex.mode(axis=1)[0]
# tmp_gex['scsplit_nogenos'] = tmp_gex.mode(axis=1)[0]

In [143]:
# tmp_atac = inter_atac.copy()
# mask = [x.split('_')[1] == samples[1] for x in tmp_atac.index]
# tmp_atac = tmp_atac[mask]
# tmp_atac['scsplit'] = tmp_atac.mode(axis=1)[0]
# tmp_atac['scsplit_nogenos'] = tmp_atac.mode(axis=1)[0]

In [144]:
# inter_gex['scsplit'] = pd.concat([scsplit_gex['donor_id'], tmp_gex['scsplit']])
# inter_atac['scsplit'] = pd.concat([scsplit_atac['donor_id'], tmp_atac['scsplit']])
# inter_gex['scsplit_nogenos'] = pd.concat([scsplit_nogenos_gex['donor_id'], tmp_gex['scsplit_nogenos']])
# inter_atac['scsplit_nogenos'] = pd.concat([scsplit_nogenos_atac['donor_id'], tmp_atac['scsplit_nogenos']])

In [145]:
# inter_gex['scsplit'].value_counts(), \
# inter_atac['scsplit'].value_counts(), \
# inter_gex['scsplit_nogenos'].value_counts(), \
# inter_atac['scsplit_nogenos'].value_counts()

In [146]:
inter_gex.to_csv(f'{projdir}/csv/demux/inter_gex.csv', sep='\t', header=True, index=True)
inter_atac.to_csv(f'{projdir}/csv/demux/inter_atac.csv', sep='\t', header=True, index=True)

In [147]:
inter_gex

,demuxlet,freemuxlet,vireo,vireo_nogenos,souporcell,souporcell_nogenos,demuxalot,scsplit,scsplit_nogenos
AAACAGCCAAGGTATA-1_20220928-IGVF-D0,27_C4,27_C4,27_C4,27_C4,27_C4,27_C4,27_C4,27_C4,27_C4
AAACAGCCAATGCGCT-1_20220928-IGVF-D0,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4
AAACAGCCAGCAAATA-1_20220928-IGVF-D0,27_C4,27_C4,27_C4,27_C4,27_C4,27_C4,27_C4,27_C4,27_C4
AAACAGCCATTGTGAT-1_20220928-IGVF-D0,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4
AAACATGCAAACATAG-1_20220928-IGVF-D0,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4
...,...,...,...,...,...,...,...,...,...
TTTGTTGGTTCCAGGG-1_20220928-IGVF-D3,26_A10,doublet,doublet,doublet,26_A10,doublet,doublet,26_A10,doublet
TTTGTTGGTTCCTGTG-1_20220928-IGVF-D3,doublet,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4
TTTGTTGGTTGCCTCA-1_20220928-IGVF-D3,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4
TTTGTTGGTTGGATAT-1_20220928-IGVF-D3,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4,25_A4
